# Model selected: **M1** SVM + DOC2VEC (DBOW)
### By **Néstor Suat** in 2020

**Descripción:** Generando el modelo entrenado con los parametros seleccionados del modelo de aprendizaje automatico **SVM** con la técnica de embedding **DBOW**.

**Input:**
* Train and Test set
* Parameters selected
    * **Preprocessing:** 6_clean_lemma_dataset_propuesta1_5050
    * **Embedding:** DBOW
    * **SVM** kernel='rbf', gamma=0.2, C=7

**Output:**
* Model output (SVM + DBOW) **Pipeline**

***

## 0. Cargando datos y limpieza

### Importando librerías

Como estamos en un archivo afuera se necesita agregar la dirección ../ (raíz del proyexto) para importar la librería de preprocesamiento.

In [1]:
import pandas as pd
import pickle

import sys
sys.path.insert(0, '../../../')

from classes.tweet2accident.preprocessing import Preprocessing
from classes.tweet2accident.doc2vec import Embedding

### Importando datasets

In [5]:
train = pd.read_csv("../../../data/v1/7030/train70.tsv", delimiter = "\t", quoting = 3)
test = pd.read_csv("../../../data/v1/7030/test30.tsv", delimiter = "\t", quoting = 3)
dataset = pd.concat([train,test])

print(train.shape, test.shape, dataset.shape) # (3804, 3)
print(train.shape[0] + test.shape[0], dataset.shape[0])

(2662, 2) (1142, 2) (3804, 2)
3804 3804


In [6]:
X, y = dataset.text, dataset.label
X_train, y_train = train.text, train.label
X_test, y_test = test.text, test.label

## 1) Preprocessing and SVM Model Parameters

In [8]:
#Preprocessing
directory = "../../../data/v1/doc2vec/"
file = "6_clean_lemma_dataset_propuesta1_5050"
type_clean = 6 #Tiene que ser el mismo que 'file' (prefijo)

#Model SVM
kernel='rbf'
gamma=0.2
C=7

## 2) Model

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

### Support Vector Machine (**DBOW + SVM**) Model
Creación de pipeline del modelo seleccionado M1: **6 Lemma + dataset propuesta1_5050 + DBOW + SVM**

In [10]:
pipeline = Pipeline([
    ('transform', Preprocessing(type_clean=type_clean, njobs=4)),
    #Tener en cuenta, que directory apunta a '../../../data/v1/doc2vec/', 
    #si queremos que el pipeline funcione en alguna ruta en especifica hay que cambiar la ruta.
    ('dbow', Embedding(type_transform='dbow',directory=directory, file=file)),
    ('clf', SVC(random_state=123, kernel=kernel, C=C, gamma=gamma))
])

In [11]:
pipeline.fit(X, y)

Pipeline(memory=None,
         steps=[('transform', Preprocessing(njobs=4, type_clean=6)),
                ('dbow',
                 Embedding(directory='../../../data/v1/doc2vec/',
                           file='6_clean_lemma_dataset_propuesta1_5050',
                           type_transform='dbow')),
                ('clf',
                 SVC(C=7, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma=0.2,
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=123, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

## 3) Evaluation

### Evaluación con **Train/test**
La partición de los datos para el conjunto de entrenamiento y pruebas consta de una relación de 70/30 correspondientemente.

In [12]:
# Predicting the Test set results
y_pred = pipeline.predict(X_test)

# Making the Confusion Matrix
cm_svm = confusion_matrix(y_test, y_pred)

metrics_svm = []
metrics = {}
metrics['accuracy'] = accuracy_score(y_test, y_pred)
metrics['recall'] = recall_score(y_test, y_pred)
metrics['precision'] = precision_score(y_test, y_pred)
metrics['f1'] = f1_score(y_test, y_pred)
metrics_svm.append(metrics)
metrics_svm = pd.DataFrame(metrics_svm)

In [13]:
print(metrics_svm)
print(cm_svm)

   accuracy    recall  precision       f1
0  0.999124  0.998221        1.0  0.99911
[[580   0]
 [  1 561]]


## 4) Save model with Pickel
Save the model to disk

In [14]:
filename = 'model_m1_dbow_svm.pkl'
with open(filename, 'wb') as model_file:
      pickle.dump(pipeline, model_file)

Load the model from disk

In [15]:
accident_clasification_model = pickle.load(open(filename, 'rb'))

In [17]:
text_test = [
    "choque entre camion y carro particular",
    "caí por accidente en tu corazón", 
    "accidente aereo deja 100 muertos en australia",
    "hola mundo",
    "accidente en carro"
]
text_predict = accident_clasification_model.predict(text_test)
print(text_predict) #El resultado es 1 si es accidente, y 0 si no lo es.

[1 0 1 0 0]
